### Greedy方法选择最优arm
(只是选，还没有concat，concat要到concat-reward.ipynb那)

In [1]:
import warnings
warnings.filterwarnings("ignore")

import re
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
import json

from DataLoader.DataLoader import DataLoader
from DataLoader.DataBasedAgent import DataBasedAgent
from DataLoader.DataRLAgent import DataRLAgent
import DeepRLAgent.VanillaInput.Train as Train
from PatternDetectionInCandleStick.Evaluation import Evaluation
import distinctipy
from datetime import datetime, timedelta
from collections import Counter

from importlib import reload

Train = reload(Train)
DeepRL = Train.Train
from utils_best_arm import add_train_portfo, add_test_portfo, plot_return, calc_return, plot_action_point, setup_logger
pd.options.display.max_colwidth = 100

device = "cpu"
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
CURRENT_PATH = os.getcwd()

In [2]:
def train(
    DATASET_NAME, 
    begin_date='2010-01-01', 
    end_date='2020-08-24', 
    split_point='2018-08-24', 
    initial_investment=1000,
    transaction_cost=0.0001,
    load_from_file=True,
    reward_type="profit",
    seed=42, 
    state_mode=1,
    n_episodes=5,
    lamb=0.0001,
    GAMMA=0.7, 
    n_step=5, 
    BATCH_SIZE=10, 
    ReplayMemorySize=20,
    TARGET_UPDATE=5,
    window_size=None, 
    train_portfolios={},
    test_portfolios={},
    arms=[],
    show_all = False,
    ratio_threshold=0.9,
):
    
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)

    data_loader = DataLoader(DATASET_NAME, begin_date=begin_date, end_date=end_date, split_point=split_point, load_from_file=load_from_file)
    dataTrain_agent = DataRLAgent(data_loader.data_train, state_mode, 'action_encoder_decoder', device, GAMMA, n_step, BATCH_SIZE, window_size, transaction_cost)
    dataTest_agent = DataRLAgent(data_loader.data_test, state_mode, 'action_encoder_decoder', device, GAMMA, n_step, BATCH_SIZE, window_size, transaction_cost)

    agent = DeepRL(data_loader, dataTrain_agent, dataTest_agent, 
                DATASET_NAME,  state_mode, window_size, transaction_cost,
                BATCH_SIZE=BATCH_SIZE, GAMMA=GAMMA, ReplayMemorySize=ReplayMemorySize,
                TARGET_UPDATE=TARGET_UPDATE, n_step=n_step, arms=arms)
    
    arm = arms[0]
    agent_test = agent.test(initial_investment=initial_investment, test_type='test', arm=arm, model_path=arm["model_path"])

    _test_portfolio = agent_test.get_daily_portfolio_value()
    test_portfolio = pd.Series(_test_portfolio).pct_change(1).fillna(0).values.tolist() 
    mean = np.mean(test_portfolio)
    std = np.std(test_portfolio)
    if std != 0:
        sharpe = mean / std
    else:
        sharpe = mean
    cumreturn = _test_portfolio[-1] / _test_portfolio[0] - 1
    arm["sharpe_list"].append(sharpe)
    arm["cumreturn_list"].append(cumreturn)
    return data_loader, arm

In [3]:
def valid(begin_date, split_date, kwargs, threshold1, threshold2, _file, seed, arms,):
    _split_point = '20{}-{}-01' 
    split_year = int(split_date.split("-")[0][-2:])
    for year in range(1):
        for month in range(1, 13):
            if month < 10:
                month = f"0{month}"
            for day in range(1, 30, 6):
                _split_point = datetime.strptime(f"20{split_year + year}-{month}-{day}", "%Y-%m-%d")
                end_date = _split_point + timedelta(days=180)
                split_point = str(_split_point.date())
                end_date = str(end_date.date())

                for arm in arms:
                    # NOTE: ts
                    # theta_hat = np.random.beta(arm["a"], arm["b"])
                    # NOTE: greedy
                    theta_hat = arm["a"] / (arm["a"] + arm["b"])
                    arm["theta"] = theta_hat

                max_index = int(sorted(arms, key=lambda x: x["theta"], reverse=True)[0]["index"])
                for arm in arms:
                    if arm["used"] < threshold1:
                        max_index = arm["index"]
                        break
                arm = arms[max_index]

                kwargs.update({
                    "begin_date": begin_date, 
                    "end_date": end_date, 
                    "split_point": split_point, 
                    "DATASET_NAME": _file,
                    "reward_type": "",
                    "seed": seed,
                    "n_episodes": 140,
                    "arms": [arm],
                    "show_all": True,
                    "ratio_threshold": 5,
                    "train_portfolios": {},
                    "test_portfolios": {},
                })

                data_loader, arm = train(**kwargs)
                
                _count = 0
                for _arm in arms:
                    if _arm["sharpe_list"].__len__() == 0: continue
                    if arm["sharpe_list"][-1] > np.mean(_arm["sharpe_list"]) and arm["cumreturn_list"][-1] > np.mean(_arm["cumreturn_list"]):
                        _count += 1
                    
                arm["used"] += 1
                
                if arm["used"] >= threshold1:
                    if _count >= threshold2:
                        arm["a"] += 1
                    else:
                        arm["b"] += 1
                else:
                    arm["a"] += 1

                # print(f"tmp: {arm['name']}-{arm['lamb']}", arm["a"], arm["b"])

    for arm in arms:
        if arm["sharpe_list"].__len__() == 0 or np.mean(arm["sharpe_list"]) == 0 or arm["a"] <= threshold1:
            arm["theta"] = 0
        else:
            arm["theta"] = arm["a"] / (arm["a"] + arm["b"])

    arms = sorted(arms, key=lambda x: x["theta"], reverse=True)
    arm = arms[0]
    return arm, end_date
    

In [ ]:
initial_investment = 1000


kwargs = {
    "load_from_file": True, 
    "transaction_cost": 0.0000,
    "initial_investment": initial_investment,
    "state_mode": 1,
    "GAMMA": 0.7, 
    "n_step": 5, 
    "BATCH_SIZE": 10, 
    "ReplayMemorySize": 20,
    "TARGET_UPDATE": 5,
    "window_size": None, 
    "lamb": 0.0,
}

_file = "AAPL"

_begin_date = '20{}-01-01'
_end_date = '20{}-01-01'
_split_point = '20{}-01-01' 

def add_arm(arms, model_path, name, lamb):
    arm = { "index": len(arms), "name": name, "theta": 0, "a": 1, "b": 1, "sharpe_list": [], "cumreturn_list": [], "lamb": lamb, "used": 0}
    arm["model_path"] = os.path.join(model_path, f'model_{arm["name"]}_{arm["lamb"]}_{seed}.pkl')
    arms.append(arm)


random_seeds = 100
threshold1, threshold2 = 2, 3
results = {}
tmp_dt = {}
files = sorted(os.listdir("./Data/"))
for _file in files[19:]:
    results2 = {}
    for seed in tqdm(range(random_seeds)):
        ls = []
        bhs = []
    
        train_portfolios = {}
        test_portfolios = {}
        
        for year in range(3):
            model_begin_date = _begin_date.format(16+year)
            model_end_date = _end_date.format(19+year)
            split_point = _split_point.format(18+year)

            arms = []
            model_path = f"Results/{_file}/{model_begin_date}~{model_end_date}/{seed}/train"
            add_arm(arms, model_path, "profit", 0)
            # add_arm(arms, model_path, "sharpe", 0.01)
            # add_arm(arms, model_path, "volatility", 10)
            add_arm(arms, model_path, "regularized", 0.01)
            add_arm(arms, model_path, "regularized", 0.05)
            add_arm(arms, model_path, "regularized", 0.1)
            # add_arm(arms, model_path, "regularized", 0.2)
            
            arm, end_date = valid(model_begin_date, split_point, kwargs, threshold1, threshold2, _file, seed, arms)
            test_start_date = end_date
            test_start_date = datetime.strptime(test_start_date, "%Y-%m-%d")
            test_end_date = test_start_date + timedelta(days=365)
            test_end_date = str(test_end_date.date())
            if not results2.get((str(test_start_date.date()), test_end_date)):
                results2[(str(test_start_date.date()), test_end_date)] = []
            results2[(str(test_start_date.date()), test_end_date)].append(f"{arm['name']}-{arm['lamb']}")

            # print(test_start_date.date(), test_end_date, f"{arm['name']}-{arm['lamb']}")
    results[_file] = results2

    for key, value in results.items():
        a = dict((':'.join(k), v) for k,v in value.items())
        tmp_dt[key] = a

    with open(f"./ts-run-results/[{random_seeds}]ts-greedy2.json", "w", encoding="utf8") as f:
        f.write(json.dumps(tmp_dt))


In [15]:
# import json

# dt = {}
# for key, value in results.items():
#     a = dict((':'.join(k), v) for k,v in value.items())
#     dt[key] = a

# with open(f"./ts-run-results/[{random_seeds}]ts-greedy.json", "r", encoding="utf8") as f:
#     dt2 = json.loads(f.read())

# dt2.update(dt)


# with open(f"./ts-run-results/[{random_seeds}]ts-greedy2.json", "w", encoding="utf8") as f:
#     f.write(json.dumps(dt2))